<a href="https://colab.research.google.com/github/tomtyiu/EpistemeAI-series-Cookbook-SDK/blob/main/deepagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip
!pip install deepagents

# uv
#uv add deepagents

# poetry
#poetry add deepagents

In [ ]:
!pip install tavily-python

In [ ]:
!pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.20 requires torchvision, which is not installed.
fastai 2.8.4 requires torchvision>=0.11, which is not installed.
fastai 2.8.4 requires torch<2.9,>=1.10, but you have torch 2.9.0 which is incompatible.


In [ ]:
!pip install -q --upgrade torch

!pip install -q transformers triton==3.4 kernels

!pip uninstall -q torchvision torchaudio -y

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.20 requires torchvision, which is not installed.
fastai 2.8.4 requires torchvision>=0.11, which is not installed.
torch 2.9.0 requires triton==3.5.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.4.0 which is incompatible.
fastai 2.8.4 requires torch<2.9,>=1.10, but you have torch 2.9.0 which is incompatible.


In [ ]:
import os
from google.colab import userdata
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')


In [ ]:
import os
from typing import Literal
from tavily import TavilyClient
from deepagents import create_deep_agent
from google.colab import userdata
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from deepagents import create_deep_agent
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()  # Or any other Store object


tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# Web search tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )


# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert code researcher. Your job is to conduct thorough research, and then write a polished report.

You have access to an internet search tool as your primary means of gathering information.

## `internet_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

# Create the deep agent
#agent = create_deep_agent(
#    tools=[internet_search],
#    system_prompt=research_instructions,
#)

llm = HuggingFacePipeline.from_model_id(
    model_id="EpistemeAI/VibeCoder-20b-RL1_0",
    task="text-generation",
    device_map="cuda",
    pipeline_kwargs=dict(
        max_new_tokens=4048,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)

model = ChatHuggingFace(llm=llm)

#model = init_chat_model(
#    model="EpistemeAI/VibeCoder-20b-RL1_0",
#)
agent = create_deep_agent(
    store=store,
    use_longterm_memory=True,
    model=model,
    tools=[internet_search],
    system_prompt=research_instructions,
)

# Invoke the agent
while prompt := input("Enter a prompt: "):
    result = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    print(result["messages"][-1].content)

#result = agent.invoke({"messages": [{"role": "user", "content": "What is langgraph?"}]})

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 64.12 MiB is free. Process 161785 has 14.68 GiB memory in use. Of the allocated memory 14.17 GiB is allocated by PyTorch, and 83.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Print the agent's response
print(result["messages"][-1].content)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-10-21

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

You have access to an internet search tool as your primary means of gathering information.

## `internet_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.


In order to complete the objective that the user asks of you, you have access to a number of standard tools.

## `write_todos`

You have access to the `write_todos` tool to help you manage and plan complex objectives.
Use this tool for complex obje